# Test of GraphSAGE
- the version of larger hidden dimension
- with `k-fold cross validation`
- use DGL
- predict `graphs`
- valid, test data are in the training dataset

In [1]:
import os
import dgl
import json
import torch
import torch as th
import dgl.nn as dglnn
# from tqdm import tqdm
from tqdm.notebook import tqdm  # 使用 notebook 版本的 tqdm
import torch.nn as nn
from dgl.nn import GraphConv, GATConv, SAGEConv
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import get_linear_schedule_with_warmup

- check the GPU and assign the GPU by the best memory usage

In [2]:
import subprocess
import torch

def get_free_gpu():
    try:
        # Run nvidia-smi command to get GPU details
        _output_to_list = lambda x: x.decode('ascii').split('\n')[:-1]
        command = "nvidia-smi --query-gpu=memory.free --format=csv,nounits,noheader"
        memory_free_info = _output_to_list(subprocess.check_output(command.split())) 
        memory_free_values = [int(x) for i, x in enumerate(memory_free_info)]
        
        # Get the GPU with the maximum free memory
        best_gpu_id = memory_free_values.index(max(memory_free_values))
        return best_gpu_id
    except:
        # If any exception occurs, default to GPU 0 (this handles cases where nvidia-smi isn't installed)
        return 0

if torch.cuda.is_available():
    # Get the best GPU ID based on free memory and set it
    best_gpu_id = get_free_gpu()
    device = torch.device(f"cuda:{best_gpu_id}")
else:
    device = torch.device("cpu")
    print("there's no available GPU")

# device = torch.device(f"cuda:{1}")
print(device)


cuda:1


## Fix the seed

In [3]:
import numpy as np
import torch
import random

#fix seed
def same_seeds(seed = 8787):
    torch.manual_seed(seed)
    # random.seed(seed) 
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

## Data Loader

In [4]:
class GraphDataset(Dataset):
    def __init__(self, data_list, device):
        self.data_list = data_list
        self.device = device

    def __len__(self):
        return len(self.data_list)
    
    def __getitem__(self, idx):
        data = self.data_list[idx]
        return data

def collate(samples):
    data_list = samples
    batched_graphs = []
    for data in data_list:
        g = dgl.graph((th.tensor(data["edge_index"][0]), th.tensor(data["edge_index"][1])), num_nodes=data["num_nodes"])

        g.ndata['feat'] = th.tensor(data["node_feat"])
        g.edata['feat'] = th.tensor(data["edge_attr"])
        g.edata['label'] = th.tensor(data["labels"])  # Add edge labels to graph

        batched_graphs.append(g)
    
    return dgl.batch(batched_graphs)

In [5]:
# datasets = ['repeat_train_320', 'valid', 'test']
datasets = ['test']
# datasets = ['valid']
dataset_data = {}

for dataset_name in tqdm(datasets):
    file_path = f"../../data_processing/dgl/data_new/exp3/training_data_repeat/transR_50/{dataset_name}.jsonl"
    
    print(file_path)
    with open(file_path) as f:
        data_list = [json.loads(line) for line in tqdm(f, position=0, leave=True)]
    
    dataset_data[dataset_name] = GraphDataset(data_list, device)
print("Datasets loaded!")

  0%|          | 0/1 [00:00<?, ?it/s]

../../data_processing/dgl/data_new/exp3/training_data_repeat/transR_50/test.jsonl


0it [00:00, ?it/s]

Datasets loaded!


- choose batch size

In [6]:
def create_dataloaders(batch_size, shuffle=True):
    dataloaders = {}
    for dataset_name, dataset in dataset_data.items():
        # do not shuffle the testing dataset
        if dataset_name == "test":
            dataloaders[dataset_name] = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=collate)    
        else:
            dataloaders[dataset_name] = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, collate_fn=collate)
    return dataloaders

dataloaders = create_dataloaders(64)

- Turn the print message to a log file

In [7]:
import datetime

now = datetime.datetime.now()

formatted_time = now.strftime("%m%d_%H:%M")

log_file_path = f"../log_message/{formatted_time}_GraphSAGE_transR_50.log"

def add_log_msg(msg, log_file_path=log_file_path):
    with open(log_file_path, 'a') as f:
        f.write(f'{datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S")}# {msg}\n')
    print(f'{datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S")}# {msg}')

print(log_file_path)

../log_message/1031_01:45_GraphSAGE_transR_50.log


### Model

In [8]:
class GraphSAGE(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim):
        super(GraphSAGE, self).__init__()
        self.layer1 = dglnn.SAGEConv(in_dim, hidden_dim, 'pool')
        self.layer2 = dglnn.SAGEConv(hidden_dim, out_dim, 'pool')
        self.dropout = nn.Dropout(0.25)

    def forward(self, g, inputs):
        h = self.layer1(g, inputs)
        h = torch.relu(h)
        h = self.dropout(h)
        h = self.layer2(g, h)
        return h

In [9]:
class MLPPredictor(nn.Module):
    def __init__(self, out_feats, out_classes):
        super().__init__()
        self.W = nn.Linear(out_feats*2, out_classes)

    def apply_edges(self, edges):
        h_u = edges.src['h']
        h_v = edges.dst['h']
        score = self.W(torch.cat([h_u, h_v], 1))
        return {'score': score}

    def forward(self, graph, h):
        with graph.local_scope():
            graph.ndata['h'] = h
            graph.apply_edges(self.apply_edges)
            return graph.edata['score']

In [10]:
class Model(nn.Module):
    def __init__(self, in_features, hidden_features, out_features, num_classes):
        super().__init__()
        self.sage = GraphSAGE(in_features, hidden_features, out_features)
        self.pred = MLPPredictor(out_features, num_classes)
      
    def forward(self, g, node_feat, return_logits=False):
        h = self.sage(g, node_feat)
        logits = self.pred(g, h)
        
        return logits

- Model Forward  

In [11]:
def model_fn(batched_g, model, criterion, device, count=1, which_type='train'):
    """Forward a batch through the model."""
#     batched_g, labels = data
    batched_g = batched_g.to(device)
    
    labels = batched_g.edata['label'].to(device)
    
    logits = model(batched_g, batched_g.ndata['feat'].float())

    loss = criterion(logits, labels)

    output = torch.softmax(logits, dim=1)
    preds = output.argmax(1)
    
    # Compute accuracy
    accuracy = torch.mean((preds == labels).float())
    
    if which_type == 'validation' and count % 1000 == 0:
        add_log_msg(f"labels of Validation: {labels} {labels.shape}")
        add_log_msg(f"predicted of Validation: {preds} {preds.shape}")
        
    elif which_type == 'test'  and count % 1000 == 0:
        add_log_msg(f"labels of Test: {labels} {labels.shape}")
        add_log_msg(f"predicted of Test: {preds} {preds.shape}")
        
    if count % 50000 == 0: 
        add_log_msg(f"labels of {count}: {labels} {labels.shape}")
        add_log_msg(f"predicted of {count}: {preds} {preds.shape}")
        
    return loss, accuracy, preds

### Training

- Fix the seed and save the model.state_dict that contains the initial weight

In [12]:
seed = 8787
same_seeds(seed)

model = Model(in_features=50, hidden_features=64, out_features=128, num_classes=167)
torch.save(model.state_dict(), 'model3_initial(graphsage)/initial_weight.pth')

In [13]:
# model.layer1.fc_self.weight
model.sage.layer1.fc_self.weight

Parameter containing:
tensor([[-0.0682,  0.0153, -0.1769,  ...,  0.0375,  0.2321, -0.2812],
        [-0.2271,  0.2290, -0.1997,  ..., -0.0095,  0.1509,  0.2686],
        [-0.2743,  0.0406, -0.1222,  ...,  0.1036, -0.1590, -0.2555],
        ...,
        [-0.0758,  0.0461,  0.1273,  ...,  0.1367,  0.0671, -0.2605],
        [-0.2425, -0.1362,  0.2474,  ..., -0.3221, -0.0595,  0.3141],
        [ 0.0234, -0.2783,  0.2146,  ..., -0.3020, -0.1751,  0.0528]],
       requires_grad=True)

- Check if model really load the model_dict

In [14]:
model = Model(in_features=50, hidden_features=64, out_features=128, num_classes=167)
model.load_state_dict(torch.load('model3_initial(graphsage)/initial_weight.pth'))
model.sage.layer1.fc_self.weight

Parameter containing:
tensor([[-0.0682,  0.0153, -0.1769,  ...,  0.0375,  0.2321, -0.2812],
        [-0.2271,  0.2290, -0.1997,  ..., -0.0095,  0.1509,  0.2686],
        [-0.2743,  0.0406, -0.1222,  ...,  0.1036, -0.1590, -0.2555],
        ...,
        [-0.0758,  0.0461,  0.1273,  ...,  0.1367,  0.0671, -0.2605],
        [-0.2425, -0.1362,  0.2474,  ..., -0.3221, -0.0595,  0.3141],
        [ 0.0234, -0.2783,  0.2146,  ..., -0.3020, -0.1751,  0.0528]],
       requires_grad=True)

In [15]:
mapping_file = './new_mapping.txt'
label_mapping = {}
with open(mapping_file, 'r') as f:
    for line in f:
        parts = line.strip().split(': ')
        label_mapping[int(parts[1])] = parts[0]

In [16]:
import csv
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from torch.optim import AdamW, lr_scheduler

seed = 8787
same_seeds(seed)

model = Model(in_features=50, hidden_features=1024, out_features=2048, num_classes=167)
best_model_path = "../checkpoint_graphSAGE/best_model_GraphSAGE_transR_50_hidden_dim_5.pt"
model = model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-4)

# T_max control the period of the lr changing -> set 1/10 first
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=36, eta_min=0, last_epoch=- 1, verbose=False)
criterion = nn.CrossEntropyLoss()

kf = KFold(n_splits=5, shuffle=False, random_state=None)
train_data = [item for item in tqdm(dataloaders['repeat_train_320'])]
kf_splits = kf.split(train_data)

total_steps = 20

# save the best model
best_val_loss = float('inf')
patience = 4  # Number of epochs with no improvement after which training will be stopped.

for fold, (train_index, val_index) in tqdm(enumerate(kf_splits)):
    
    train_set = [train_data[i] for i in train_index]
    val_set = [train_data[i] for i in val_index]
    
    # 初始化模型和优化器
    model = Model(in_features=50, hidden_features=1024, out_features=2048, num_classes=167)
    model = model.to(device)
    optimizer = AdamW(model.parameters(), lr=5e-4)
    scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=36, eta_min=0, last_epoch=- 1, verbose=False)
    
    waiting = 0  # The number of epochs with no improvement so far.
    early_stop = False

    for epoch in tqdm(range(total_steps)):
        # Train
        model.train()
        total_loss = 0.0
        total_accuracy = 0.0
        num_batches = 0

        for batched_g in tqdm(train_set, desc="Training", position=0, leave=True):
            num_batches += 1
            loss, accuracy, _ = model_fn(batched_g, model, criterion, device, num_batches, which_type='train')

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            total_accuracy += accuracy.item()


        add_log_msg(f"total batches: {num_batches}")

        avg_loss = total_loss / num_batches
        avg_accuracy = total_accuracy / num_batches

        add_log_msg(f'Epoch {epoch} | Train Loss: {avg_loss:.4f} | Train Accuracy: {avg_accuracy:.4f}')

        # Validation Part
        model.eval()
        total_accuracy = 0.0
        total_loss = 0.0
        num_batches = 0

        with torch.no_grad():
            for batched_g in tqdm(val_set, desc="Validation", position=0, leave=True):
                loss, accuracy, _ = model_fn(batched_g, model, criterion, device, num_batches, which_type='validation')
                total_accuracy += accuracy.item()
                total_loss += loss.item()
                num_batches += 1

        avg_accuracy = total_accuracy / num_batches
        current_loss = total_loss / num_batches

        add_log_msg(f'Validation Loss: {current_loss:.4f} | Validation Accuracy: {avg_accuracy:.4f}\n')


        if current_loss < best_val_loss:
            best_val_loss = current_loss
            waiting = 0

            if os.path.exists(best_model_path):
                os.remove(best_model_path)
                add_log_msg("Find a better model!!")

            torch.save(model.state_dict(), best_model_path)

        else:
            waiting += 1
            if waiting >= patience:
                early_stop = True

        if epoch == total_steps - 1 or early_stop == True:  # Last epoch
            pretrained_model_path = '../checkpoint_graphSAGE/best_model_GraphSAGE_transR_50_hidden_dim_5.pt'
            model.load_state_dict(torch.load(pretrained_model_path))
            model.to(device)

            true_labels = []
            predicted_labels = []

            with torch.no_grad():
                for batched_g in tqdm(train_set, desc="Generating Train Report", position=0, leave=True):
                    loss, accuracy, predicted = model_fn(batched_g, model, criterion, device, which_type='train')
                    labels = batched_g.edata['label'].to(device)

                    true_labels.extend(labels.cpu().numpy())
                    predicted_labels.extend(predicted.cpu().numpy())

            # Assuming that the mapping and other things are defined earlier or globally
            training_mapped_true_labels = [label_mapping[label] for label in true_labels]
            training_mapped_predicted_labels = [label_mapping[label] for label in predicted_labels]

            # Generate classification report
            train_report = classification_report(training_mapped_true_labels, training_mapped_predicted_labels)
            add_log_msg(f"Train Classification Report at Epoch {epoch}:\n{train_report}")

        if early_stop:
            add_log_msg("============================== Early stopping ==================================")
            break

KeyError: 'repeat_train_320'

In [17]:
model.load_state_dict(torch.load(best_model_path))
model.to(device)

true_labels = []
predicted_labels = []

with torch.no_grad():
    for batched_g in tqdm(dataloaders['repeat_train_320'], desc="Generating Train Report", position=0, leave=True):
        loss, accuracy, predicted = model_fn(batched_g, model, criterion, device, which_type='train')
        labels = batched_g.edata['label'].to(device)

        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())

# Assuming that the mapping and other things are defined earlier or globally
mapped_true_labels = [label_mapping[label] for label in true_labels]
mapped_predicted_labels = [label_mapping[label] for label in predicted_labels]

# Generate classification report
train_report = classification_report(mapped_true_labels, mapped_predicted_labels)
add_log_msg(f"Train Classification Report at Epoch {epoch}:\n{train_report}")

KeyError: 'repeat_train_320'

In [22]:
train_report

'                                                precision    recall  f1-score   support\n\nT1003.001_0ef4cc7b-611c-4237-b20b-db36b6906554       1.00      1.00      1.00     41664\n    T1003.001_35d92515122effdd73801c6ac3021da7       1.00      1.00      1.00      3822\n    T1003.002_5a484b65c247675e3b7ada4ba648d376       1.00      1.00      1.00      3170\n    T1003.002_7fa4ea18694f2552547b65e23952cabb       1.00      1.00      1.00      9600\n    T1003.003_9f73269695e54311dd61dc68940fb3e1       0.99      1.00      1.00    204857\n    T1003.003_f049b89533298c2d6cd37a940248b219       1.00      0.99      1.00    204717\n        T1003_18f31c311ac208802e88ab8d5af8603e       1.00      1.00      1.00      3828\n        T1007_9d03c91bdae5a80f17f89c987942b5a8       1.00      1.00      1.00      3744\n    T1007_c6607391-d02c-44b5-9b13-d3492ca58599       0.97      0.96      0.96    205113\n        T1007_d6bb2a19da7246731ed9c44831b135f8       0.08      0.00      0.00      1935\n    T1016_14a21534

In [23]:
train_report = classification_report(mapped_true_labels, mapped_predicted_labels, output_dict=True)
report_df = pd.DataFrame(train_report).transpose()

report_df

NameError: name 'mapped_true_labels' is not defined

In [ ]:
report_df.to_csv('hidden_dim_4.csv', index=True)

report_df

In [ ]:
filtered_report = report_df[(report_df['support'] == 256000)] # 256000 = 100 x 8 x 320 -> single triplet ones
num_rows = filtered_report.shape[0]
print("Number of support=1600:", num_rows)

filtered_report = report_df[(report_df['f1-score'] <= 0.2) & (report_df['support'] == 256000)]
num_rows = filtered_report.shape[0]
print("Number of support=100 and f1-score<=0.2:", num_rows)

filtered_report = report_df[(report_df['f1-score'] > 0.5) & (report_df['support'] == 256000)]
num_rows = filtered_report.shape[0]
print("Number of support=100 and f1-score<=0.2:", num_rows)

filtered_report = report_df[(report_df['f1-score'] <= 0.5) & (report_df['support'] == 256000)]
num_rows = filtered_report.shape[0]
print("Number of support=100 and f1-score<=0.2:", num_rows)

In [ ]:
import matplotlib.pyplot as plt

filtered_report = report_df[(report_df['support'] == 256000)]
ax = filtered_report[['f1-score']].plot(kind='bar', figsize=(22,12))

# 添加 f1-score 值到 bar 上方
for p in ax.patches:
    ax.annotate(format(p.get_height(), '.2f'), 
                (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha = 'center', 
                va = 'center', 
                xytext = (0, 9), 
                textcoords = 'offset points')

plt.show()

In [ ]:
threshold = 0.2
filtered_report = report_df[report_df['f1-score'] <= threshold]

filtered_report

In [ ]:
filtered_report = report_df[(report_df['f1-score'] == 0) & (report_df['support'] == 100)]

num_rows = filtered_report.shape[0]
print("Number of rows:", num_rows)

# labels_indices = filtered_report.index.tolist()
# # Print to check if the indices are what we expect
# print("Filtered Report Indices:", labels_indices)

# mapped_labels = [key for label in labels_indices for key, value in label_mapping.items() if value == label]

# with open('triplets_1.txt', 'w') as f:
#     for label in mapped_labels:
#         f.write(f'{label}\n')

# print(f'{len(labels_indices)} labels have been written to filtered_labels.txt')


filtered_report

### test of valid and test part is ``graph``

- 60 APs in training x 10000times
- 5 APs in validation x 4 times
- 3 APs in test x 4 times
- Batch size = 4

In [25]:
model.load_state_dict(torch.load(best_model_path))

model.to(device)
model.eval()

total = 0
correct = 0
count = 0

true_labels = []
predicted_labels = []

with torch.no_grad():
    for batched_g in tqdm(dataloaders['test'], desc="Testing", position=0, leave=True):
#         print(f"data:{data[1]}")
        loss, accuracy, predicted = model_fn(batched_g, model, criterion, device, count, which_type='test')
        labels = batched_g.edata['label'].to(device)
        
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())
        
        if count % 5000 == 0:
            add_log_msg(f"labels: {labels} {labels.shape}")
            add_log_msg(f"predicted: {predicted} {predicted.shape}")
            
        count += 1
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

add_log_msg(f'Test Accuracy: {100 * correct / total} %\n\n\n')


# ======================================== handlig the output excel files ========================================
mapped_true_labels = [label_mapping[label] for label in true_labels]
mapped_predicted_labels = [label_mapping[label] for label in predicted_labels]

# 生成Scikit-learn报告信息的DataFrame
report_data = classification_report(mapped_true_labels, mapped_predicted_labels, output_dict=True)
report_df = pd.DataFrame(report_data).transpose()

report_folder = 'classification_report'
os.makedirs(report_folder, exist_ok=True)

count = 0
while True:
    report_filename = f'classification_report-transE_50-graphSAGE-{count}.xlsx'
    labels_filename = f'mapped_true_predicted_labels-transE_50-graphSAGE-{count}.xlsx'
    
    report_path = os.path.join(report_folder, report_filename)
    labels_path = os.path.join(report_folder, labels_filename)
    
    if not os.path.exists(report_path) and not os.path.exists(labels_path):
        break
    count += 1

    
report_df.to_excel(report_path, index_label='Label')

mapped_labels_df = pd.DataFrame({'true_label': mapped_true_labels, 'predicted_label': mapped_predicted_labels})
mapped_labels_df.to_excel(labels_path, index=False)

add_log_msg(f"report path: {report_path}")
add_log_msg(f"label path: {labels_path}")

mapped_report = classification_report(mapped_true_labels, mapped_predicted_labels)
add_log_msg(f"mapped_report:\n{mapped_report}")

Testing:   0%|          | 0/258 [00:00<?, ?it/s]

10/27/2023, 15:50:51# labels of Test: tensor([65, 65, 65,  ..., 70, 70, 70], device='cuda:1') torch.Size([2718])
10/27/2023, 15:50:51# predicted of Test: tensor([65, 65, 65,  ..., 70, 70, 70], device='cuda:1') torch.Size([2718])
10/27/2023, 15:50:51# labels of 0: tensor([65, 65, 65,  ..., 70, 70, 70], device='cuda:1') torch.Size([2718])
10/27/2023, 15:50:51# predicted of 0: tensor([65, 65, 65,  ..., 70, 70, 70], device='cuda:1') torch.Size([2718])
10/27/2023, 15:50:51# labels: tensor([65, 65, 65,  ..., 70, 70, 70], device='cuda:1') torch.Size([2718])
10/27/2023, 15:50:51# predicted: tensor([65, 65, 65,  ..., 70, 70, 70], device='cuda:1') torch.Size([2718])
10/27/2023, 15:51:00# Test Accuracy: 97.0018983894309 %





/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


10/27/2023, 15:51:30# report path: classification_report/classification_report-transE_50-graphSAGE-5.xlsx
10/27/2023, 15:51:30# label path: classification_report/mapped_true_predicted_labels-transE_50-graphSAGE-5.xlsx


/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


10/27/2023, 15:51:42# mapped_report:
                                                precision    recall  f1-score   support

T1003.001_0ef4cc7b-611c-4237-b20b-db36b6906554       1.00      1.00      1.00      6400
    T1003.001_35d92515122effdd73801c6ac3021da7       1.00      1.00      1.00       600
    T1003.002_5a484b65c247675e3b7ada4ba648d376       1.00      1.00      1.00       500
    T1003.002_7fa4ea18694f2552547b65e23952cabb       1.00      1.00      1.00      1500
    T1003.003_9f73269695e54311dd61dc68940fb3e1       0.03      0.04      0.03       100
    T1003.003_f049b89533298c2d6cd37a940248b219       0.03      0.03      0.03       100
        T1003_18f31c311ac208802e88ab8d5af8603e       1.00      1.00      1.00       600
        T1007_9d03c91bdae5a80f17f89c987942b5a8       1.00      1.00      1.00       600
    T1007_c6607391-d02c-44b5-9b13-d3492ca58599       0.01      0.01      0.01       100
        T1007_d6bb2a19da7246731ed9c44831b135f8       0.00      0.00      0.00     

/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
model.load_state_dict(torch.load(best_model_path))

model.to(device)
model.eval()

total = 0
correct = 0
count = 0

true_labels = []
predicted_labels = []

with torch.no_grad():
    for batched_g in tqdm(dataloaders['test'], desc="Testing", position=0, leave=True):
#         print(f"data:{data[1]}")
        loss, accuracy, predicted = model_fn(batched_g, model, criterion, device, count, which_type='test')
        labels = batched_g.edata['label'].to(device)
        
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())
        
        if count % 5000 == 0:
            add_log_msg(f"labels: {labels} {labels.shape}")
            add_log_msg(f"predicted: {predicted} {predicted.shape}")
            
        count += 1
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

add_log_msg(f'Test Accuracy: {100 * correct / total} %\n\n\n')


# ======================================== handlig the output excel files ========================================
mapped_true_labels = [label_mapping[label] for label in true_labels]
mapped_predicted_labels = [label_mapping[label] for label in predicted_labels]

# 生成Scikit-learn报告信息的DataFrame
report_data = classification_report(mapped_true_labels, mapped_predicted_labels, output_dict=True)
report_df = pd.DataFrame(report_data).transpose()

report_folder = 'classification_report'
os.makedirs(report_folder, exist_ok=True)

count = 0
while True:
    report_filename = f'classification_report-transE_50-graphSAGE-{count}.xlsx'
    labels_filename = f'mapped_true_predicted_labels-transE_50-graphSAGE-{count}.xlsx'
    
    report_path = os.path.join(report_folder, report_filename)
    labels_path = os.path.join(report_folder, labels_filename)
    
    if not os.path.exists(report_path) and not os.path.exists(labels_path):
        break
    count += 1

    
report_df.to_excel(report_path, index_label='Label')

mapped_labels_df = pd.DataFrame({'true_label': mapped_true_labels, 'predicted_label': mapped_predicted_labels})
mapped_labels_df.to_excel(labels_path, index=False)

add_log_msg(f"report path: {report_path}")
add_log_msg(f"label path: {labels_path}")

mapped_report = classification_report(mapped_true_labels, mapped_predicted_labels)
add_log_msg(f"mapped_report:\n{mapped_report}")

Testing:   0%|          | 0/258 [00:00<?, ?it/s]

10/31/2023, 01:47:37# labels of Test: tensor([65, 65, 65,  ..., 70, 70, 70], device='cuda:1') torch.Size([2718])
10/31/2023, 01:47:37# predicted of Test: tensor([65, 65, 65,  ..., 70, 70, 70], device='cuda:1') torch.Size([2718])
10/31/2023, 01:47:37# labels of 0: tensor([65, 65, 65,  ..., 70, 70, 70], device='cuda:1') torch.Size([2718])
10/31/2023, 01:47:37# predicted of 0: tensor([65, 65, 65,  ..., 70, 70, 70], device='cuda:1') torch.Size([2718])
10/31/2023, 01:47:37# labels: tensor([65, 65, 65,  ..., 70, 70, 70], device='cuda:1') torch.Size([2718])
10/31/2023, 01:47:37# predicted: tensor([65, 65, 65,  ..., 70, 70, 70], device='cuda:1') torch.Size([2718])
10/31/2023, 01:47:46# Test Accuracy: 97.0018983894309 %





/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


10/31/2023, 01:48:20# report path: classification_report/classification_report-transE_50-graphSAGE-6.xlsx
10/31/2023, 01:48:20# label path: classification_report/mapped_true_predicted_labels-transE_50-graphSAGE-6.xlsx


/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


10/31/2023, 01:48:32# mapped_report:
                                                precision    recall  f1-score   support

T1003.001_0ef4cc7b-611c-4237-b20b-db36b6906554       1.00      1.00      1.00      6400
    T1003.001_35d92515122effdd73801c6ac3021da7       1.00      1.00      1.00       600
    T1003.002_5a484b65c247675e3b7ada4ba648d376       1.00      1.00      1.00       500
    T1003.002_7fa4ea18694f2552547b65e23952cabb       1.00      1.00      1.00      1500
    T1003.003_9f73269695e54311dd61dc68940fb3e1       0.03      0.04      0.03       100
    T1003.003_f049b89533298c2d6cd37a940248b219       0.03      0.03      0.03       100
        T1003_18f31c311ac208802e88ab8d5af8603e       1.00      1.00      1.00       600
        T1007_9d03c91bdae5a80f17f89c987942b5a8       1.00      1.00      1.00       600
    T1007_c6607391-d02c-44b5-9b13-d3492ca58599       0.01      0.01      0.01       100
        T1007_d6bb2a19da7246731ed9c44831b135f8       0.00      0.00      0.00     

/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


- Get the ROC_AUC score and the confusion matrix

In [25]:
from sklearn.metrics import roc_auc_score, confusion_matrix

# # 计算 One-vs-Rest AUC
# n_classes = len(np.unique(true_labels))

# # 将标签转换成独热编码格式
# from sklearn.preprocessing import label_binarize
# true_labels_bin = label_binarize(true_labels, classes=np.arange(n_classes))

# # 计算 One-vs-Rest AUC
# auc_scores = roc_auc_score(true_labels_bin, predicted_labels, average='macro')
# add_log_msg(f'Macro-Average One-vs-Rest AUC: {auc_scores}')


# 计算混淆矩阵
conf_matrix = confusion_matrix(true_labels, predicted_labels)
# add_log_msg(f'Confusion Matrix:\n{conf_matrix}')

with open('confusion_matrix.txt', 'w') as f:
    np.savetxt(f, conf_matrix, fmt='%d')

    print("Done")



Done


In [ ]:
import csv
import pandas as pd
from sklearn.metrics import classification_report
from torch.optim import AdamW, lr_scheduler

seed = 5269
same_seeds(seed)

model = Model(in_features=50, hidden_features=1024, out_features=2048, num_classes=167)
# in_dim means the dimension of the node_feat(50 dim, since the 50-dim embedding)
# out_dim means the # of the categories -> 168 for out tasks
# model.load_state_dict(torch.load('model3_initial(graphsage)/initial_weight.pth'))
best_model_path = "../checkpoint_graphSAGE/best_model_GraphSAGE_transR_50_hidden_dim_4.pt"

model = model.to(device)

# optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
optimizer = AdamW(model.parameters(), lr=5e-4)
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=2, eta_min=0, last_epoch=- 1, verbose=False)
criterion = nn.CrossEntropyLoss()

total_steps = 20

# save the best model
best_val_loss = float('inf')
patience = 20  # Number of epochs with no improvement after which training will be stopped.
waiting = 0  # The number of epochs with no improvement so far.
early_stop = False

# Training Part
for epoch in tqdm(range(total_steps)):
    # Train
    model.train()
    total_loss = 0.0
    total_accuracy = 0.0
    num_batches = 0
        
    for batched_g in tqdm(dataloaders['repeat_train_320'], desc="Training", position=0, leave=True):
        num_batches += 1
        loss, accuracy, _ = model_fn(batched_g, model, criterion, device, num_batches, which_type='train')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_accuracy += accuracy.item()

    scheduler.step()
    add_log_msg(f"total batches: {num_batches}")

    avg_loss = total_loss / num_batches
    avg_accuracy = total_accuracy / num_batches
    add_log_msg(f'Epoch {epoch} | Train Loss: {avg_loss:.4f} | Train Accuracy: {avg_accuracy:.4f}')

    model.eval()
    total_accuracy = 0.0
    total_loss = 0.0
    num_batches = 0


    with torch.no_grad():
        for batched_g in tqdm(dataloaders['valid'], desc="Validation", position=0, leave=True):
            loss, accuracy, _ = model_fn(batched_g, model, criterion, device, num_batches, which_type='validation')
            total_accuracy += accuracy.item()
            total_loss += loss.item()
            num_batches += 1

    avg_accuracy = total_accuracy / num_batches
    current_loss = total_loss / num_batches
    
    add_log_msg(f'Validation Loss: {current_loss:.4f} | Validation Accuracy: {avg_accuracy:.4f}\n')
    
    if current_loss < best_val_loss:
        best_val_loss = current_loss
        waiting = 0
        
        if os.path.exists(best_model_path):
            os.remove(best_model_path)
            add_log_msg("Find a better model!!")
        torch.save(model.state_dict(), best_model_path)

    else:
        waiting += 1
        if waiting >= patience:
            early_stop = True
            
    if epoch == total_steps - 1 or early_stop == True:  # Last epoch
        pretrained_model_path = '../checkpoint_graphSAGE/best_model_GraphSAGE_transR_50_hidden_dim_4.pt'
        model.load_state_dict(torch.load(pretrained_model_path))
        model.to(device)
        
        true_labels = []
        predicted_labels = []

        with torch.no_grad():
            for batched_g in tqdm(dataloaders['repeat_train_320'], desc="Generating Train Report", position=0, leave=True):
                loss, accuracy, predicted = model_fn(batched_g, model, criterion, device, which_type='train')
                labels = batched_g.edata['label'].to(device)
                
                true_labels.extend(labels.cpu().numpy())
                predicted_labels.extend(predicted.cpu().numpy())

        # Assuming that the mapping and other things are defined earlier or globally
        training_mapped_true_labels = [label_mapping[label] for label in true_labels]
        training_mapped_predicted_labels = [label_mapping[label] for label in predicted_labels]

        # Generate classification report
        train_report = classification_report(training_mapped_true_labels, training_mapped_predicted_labels)
        add_log_msg(f"Train Classification Report at Epoch {epoch}:\n{train_report}")

    
    if early_stop:
        add_log_msg("============================== Early stopping ==================================")
        break
        

  0%|          | 0/20 [00:00<?, ?it/s]

Training:   0%|          | 0/213400 [00:00<?, ?it/s]

10/20/2023, 17:19:25# labels of 50000: tensor([ 55, 157,   9, 150, 112,  60, 116,  30,  34, 143, 125,  18,  48,  83,
        164, 163,  47, 163, 121, 142,  30, 151, 143, 124,  74, 111,  47,  34,
        119, 142,   2,  83, 125,  48,  31,  53,  57,  36,  74,  60,  76, 164,
         14, 112,   1, 121,   2,  53, 125, 152, 124,  92,  81,  81,  97,  34,
         97,  30,  87,  57,   2,  81,  58,  58,  58,  58,  58,  58,  58, 163],
       device='cuda:0') torch.Size([70])
10/20/2023, 17:19:25# predicted of 50000: tensor([ 36, 164, 162, 124, 112,  87,  54,  57, 151,  33, 119, 152,  49,  30,
         44,  49,  11,  31,  38, 142,  11, 109,  30,  54, 124,  30,  33, 124,
        119,  54, 162,  49, 124,  11, 157, 124,  57,  36, 119,  54,  87, 111,
         38,  33, 150,  83, 162, 162, 124,  57, 124,  30, 125,  12,  54,  14,
        157,  81,  74, 112,  83,  54,  58,  58,  58,  58,  58,  58,  58,  30],
       device='cuda:0') torch.Size([70])
10/20/2023, 17:47:03# labels of 100000: tensor([112, 14

Validation:   0%|          | 0/258 [00:00<?, ?it/s]

10/20/2023, 18:53:10# labels of Validation: tensor([  9, 143,  27,  27,  27,  41,  41,  41,  41,  41, 116, 150, 130, 130,
        130, 130, 130,  35,  35,  35,  35,  35,  35,  89,  89,  37,  37,  37,
         37,  37,  60, 139, 139, 139, 139, 139,  29,  29,  29,  29,  29,  29,
         29,  29,  29,  29, 129, 129, 129, 129, 129, 129, 129,  52,  52,  52,
         52,  52,  52,  52,  52,  52,  52,  52,  65,  52,  52,  52,  52,  52,
         52,  52,  52,  52,  52,  52,  52,  52,  52,  52,  52,  52,  52,  52,
         52,  52,  52,  52,  52,  52,  52,  52,  52,  52,  52,  52,  52,  52,
         52,  52,  52,  52,  52,  52,  52,  52,  52,  52,  52,  52,  52,  52,
         52,  52,  52,  52,  52,  52,  52,  52,  52,  52,  52,  52,  52,  52,
         52,  52,  52,  52,  52,  52,  52,  52,  52,  52,  52,  52,  52,  52,
         52,  52,  52,  52,  52,  52,  52,  52,  52,  52,  52,  52,  52,  52,
         52,  52,  52,  52,  52,  52,  52,  52,  52,  52,  52,  52,  52,  52,
         52,  52,  5

...

Validation:   0%|          | 0/258 [00:00<?, ?it/s]

10/21/2023, 11:21:08# labels of Validation: tensor([ 57, 130, 130, 130, 130, 130,  65,  65,  65,  65,  65,  65,  65,  65,
         65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,
         65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,
         65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,
         65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,
         65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,
         65,  65,  65,  65,  65,  65,  65,  65, 141, 141, 141, 141, 141, 141,
        141, 141, 141, 141,  72,  72,  72,  72, 117, 117, 117,  95,  95,  95,
         65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,
         65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  70,
         70,  70,  70,  70,  70,  70,  70,  70,  70,  70,  70,  70,  70,  70,
         70,  70,  70,  70,  70,  70,  70,  70,  17,  17,  17,  17,  17,  17,
         17,  17,  1

10/21/2023, 11:21:17# Validation Loss: 0.3177 | Validation Accuracy: 0.9569



Training:   0%|          | 0/213400 [00:00<?, ?it/s]

10/21/2023, 11:49:33# labels of 50000: tensor([112,  48, 104,  81,  76, 143, 150,  53,  33,  60,  55,  42,  31,  55,
        109, 164, 121, 111,  60, 121,  42, 158,   9,  76,  12,  34,  74,  48,
         18,  47,   9,  42,  12, 121,  74, 158,  48,  97,  81, 109, 116,  33,
         87,  76,  31,  55,   2, 163, 104, 124, 112, 158,  38,  47, 142,   4,
         33,  48,  87, 111,  57,  48, 104, 111], device='cuda:0') torch.Size([64])
10/21/2023, 11:49:33# predicted of 50000: tensor([112,  48, 143,  81,  76, 143, 150,  53,  33,  60,  55,  42,  31,  55,
        109, 164, 121,  47,  60, 121,  42, 158,  14,  76,  12,  34,  74,  48,
         18,  47,   9,  42,  12, 121,  74, 158,  48,  97,  76, 109, 163,  33,
         87,  76,  31,  55,   2, 163,  34, 116, 112, 158,  38,  47, 142,   4,
         24,  48,  36, 111,  57,  31, 104, 111], device='cuda:0') torch.Size([64])
10/21/2023, 12:19:41# labels of 100000: tensor([111,  14, 158,  44,  30, 121, 152,   9, 111,   4,  34,  34,  24, 116,
        158

In [ ]:
model.load_state_dict(torch.load(best_model_path))

model.to(device)
model.eval()

total = 0
correct = 0
count = 0

true_labels = []
predicted_labels = []

with torch.no_grad():
    for batched_g in tqdm(dataloaders['test'], desc="Testing", position=0, leave=True):
#         print(f"data:{data[1]}")
        loss, accuracy, predicted = model_fn(batched_g, model, criterion, device, count, which_type='test')
        labels = batched_g.edata['label'].to(device)
        
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())
        
        if count % 5000 == 0:
            add_log_msg(f"labels: {labels} {labels.shape}")
            add_log_msg(f"predicted: {predicted} {predicted.shape}")
            
        count += 1
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

add_log_msg(f'Test Accuracy: {100 * correct / total} %\n\n\n')


# ======================================== handlig the output excel files ========================================
mapped_true_labels = [label_mapping[label] for label in true_labels]
mapped_predicted_labels = [label_mapping[label] for label in predicted_labels]

# 生成Scikit-learn报告信息的DataFrame
report_data = classification_report(mapped_true_labels, mapped_predicted_labels, output_dict=True)
report_df = pd.DataFrame(report_data).transpose()

report_folder = 'classification_report'
os.makedirs(report_folder, exist_ok=True)

count = 0
while True:
    report_filename = f'classification_report-transE_50-graphSAGE-{count}.xlsx'
    labels_filename = f'mapped_true_predicted_labels-transE_50-graphSAGE-{count}.xlsx'
    
    report_path = os.path.join(report_folder, report_filename)
    labels_path = os.path.join(report_folder, labels_filename)
    
    if not os.path.exists(report_path) and not os.path.exists(labels_path):
        break
    count += 1

    
report_df.to_excel(report_path, index_label='Label')

mapped_labels_df = pd.DataFrame({'true_label': mapped_true_labels, 'predicted_label': mapped_predicted_labels})
mapped_labels_df.to_excel(labels_path, index=False)

add_log_msg(f"report path: {report_path}")
add_log_msg(f"label path: {labels_path}")

mapped_report = classification_report(mapped_true_labels, mapped_predicted_labels)
add_log_msg(f"mapped_report:\n{mapped_report}")